# Tiền xử lý dữ liệu

## Import thư viện và load data

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

pd.options.display.float_format = '{:,.2f}'.format

In [35]:
df = pd.read_csv('../data/players_data.csv')
print(f'Kích thước dữ liệu: {df.shape}')
df.head(5)

Kích thước dữ liệu: (32601, 20)


,name,age_at_last_season,country_of_birth,country_of_citizenship,position,sub_position,foot,height_in_cm,current_club_name,current_club_domestic_competition_id,club_position,last_season,contract_expiration_date,market_value_in_eur,agent_name,total_goals,total_assists,total_minutes_played,total_yellow_cards,total_red_cards
0,Miroslav Klose,37.00,Poland,Germany,Attack,Centre-Forward,right,184.00,Società Sportiva Lazio S.p.A.,serie-a,8.00,2015,NaN,"1,000,000.00",ASBW Sport Marketing,12,8,2429,6,0
1,Roman Weidenfeller,37.00,Germany,Germany,Goalkeeper,Goalkeeper,left,190.00,Borussia Dortmund,bundesliga,4.00,2017,NaN,"750,000.00",Neubauer 13 GmbH,0,0,181,0,0
2,Dimitar Berbatov,34.00,Bulgaria,Bulgaria,Attack,Centre-Forward,NaN,NaN,Panthessalonikios Athlitikos Omilos Konstantin...,super-league-1,NaN,2015,NaN,"1,000,000.00",CSKA-AS-23 Ltd.,6,0,1656,0,0
3,Lúcio,34.00,Brazil,Brazil,Defender,Centre-Back,NaN,NaN,Juventus Football Club,serie-a,1.00,2012,NaN,"200,000.00",NaN,0,0,307,0,0
4,Tom Starke,36.00,East Germany (GDR),Germany,Goalkeeper,Goalkeeper,right,194.00,FC Bayern München,bundesliga,NaN,2017,NaN,"100,000.00",IFM,0,0,450,0,0


## Data cleaning
Dataset là màn hồ sơ, số liệu thống kê và giá trị của một cầu thủ tại mùa giải gần nhất. Nên loại bỏ các cầu thủ không thi đấu tại mùa giải gần nhất (số phút thi đấu = 0). Ngoài ra loại bỏ cả các cầu thủ không có giá trị chuyển nhượng và những giá trị phi lý (như chiều cao <100cm)

In [36]:
miss_vals = ["Missing", "missing", "Unknown", "unknown", "N/A", "n/a", "?", ""]
df.replace(miss_vals, np.nan, inplace=True)

# Xử lý Target rỗng
df = df.dropna(subset=['market_value_in_eur'])

# Xử lý cầu thủ không thi đấu
df = df[df['total_minutes_played'] > 0]

# Xóa chiều cao vô lý 
df = df[df['height_in_cm'] >= 150]

# Xóa những cầu thủ có hợp đống kết thúc trước mùa giải gần nhất thi đấu
df['contract_expiration_date'] = pd.to_datetime(df['contract_expiration_date'], errors='coerce')
contract_year = df['contract_expiration_date'].dt.year
df = df[
    (contract_year >= df['last_season']) | 
    (pd.isna(contract_year))
]

# Xóa những cầu thủ thiếu thông tin vị trí, vì đây là thông tin cốt lõi không thể thay thế
df = df.dropna(subset=['position'])

print(f"Dữ liệu sau khi làm sạch: {df.shape}")

Dữ liệu sau khi làm sạch: (21378, 20)


## Xử lý missing values

In [37]:
# --- 1. Cột dạng Số (Numerical) ---
# Điền missing bằng Median (Trung vị) để tránh ảnh hưởng bởi outlier
num_cols = df.select_dtypes(include=[np.number]).columns
for col in num_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

# --- 2. Cột dạng Phân loại (Categorical) ---
# Cột 'foot': Điền bằng giá trị phổ biến nhất (Mode)
if df['foot'].isnull().sum() > 0:
    df['foot'] = df['foot'].fillna(df['foot'].mode()[0])

# Cột 'sub_position': Điền logic dựa theo 'position' cha
# Tạo từ điển map: {Position -> Mode của Sub-position đó}
pos_map = df.groupby('position')['sub_position'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').to_dict()

# Áp dụng map vào những dòng bị thiếu sub_position
df['sub_position'] = df.apply(
    lambda row: pos_map.get(row['position'], 'Unknown') if pd.isna(row['sub_position']) else row['sub_position'],
    axis=1
)

df['country_of_citizenship'] = df['country_of_citizenship'].fillna(df['country_of_birth'])
df['country_of_citizenship'] = df['country_of_citizenship'].fillna('Unknown')

# Các cột categorical còn lại: Điền "Unknown"
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna('Unknown')

print("Đã xử lý xong Missing Values.")

Đã xử lý xong Missing Values.


### Feature engineering
Tạo ra những cột mới có ý nghĩa cho mô hình. Bao gồm có người đại diện không. Số bàn thắng + kiến tạo mỗi 90 phút. Nếu cầu thủ đó ra sân it hơn 90 phút, số G+A/90 sẽ được tính tính bằng đúng số bàn thắng và số kiến tạo của cầu thủ đó

In [38]:
import numpy as np

# 1. Has Agent
df['has_agent'] = df['agent_name'].apply(lambda x: 0 if x == 'Unknown' else 1)

# 2. Goals + Assists per 90 min (Logic xử lý cầu thủ đá ít < 90p)
df['ga_per90min'] = np.where(
    df['total_minutes_played'] < 90,
    df['total_goals'] + df['total_assists'], # Lấy giá trị tuyệt đối nếu đá ít
    (df['total_goals'] + df['total_assists']) / df['total_minutes_played'] * 90
)
df['ga_per90min'] = df['ga_per90min'].round(2)

# 3. Thời hạn hợp đồng (Logic mới: Năm hết hạn - Mùa giải trước)
df['contract_expiration_date'] = pd.to_datetime(df['contract_expiration_date'], errors='coerce')
df['contract_years_remaining'] = df['contract_expiration_date'].dt.year - df['last_season']
median_years = df['contract_years_remaining'].median()
df['contract_years_remaining'] = df['contract_years_remaining'].fillna(median_years)

df['contract_years_remaining'] = df['contract_years_remaining'].astype(int)

print("Đã tạo xong các Feature (Contract Years = Expiry Year - Last Season).")
print(df[['last_season', 'contract_expiration_date', 'contract_years_remaining']].head())

Đã tạo xong các Feature (Contract Years = Expiry Year - Last Season).
   last_season contract_expiration_date  contract_years_remaining
0         2015                      NaT                         3
1         2017                      NaT                         3
4         2017                      NaT                         3
7         2015                      NaT                         3
9         2015               2023-12-31                         8


## Lưu dữ liệu để sử dụng cho trả lời câu hỏi

In [39]:
df.to_csv('../data/processed_players_data.csv', index=False)

### Encoding

In [40]:
numeric_features = [
    'age_at_last_season', 'height_in_cm', 'club_position',
    'total_goals', 'total_assists', 'total_minutes_played',
    'total_yellow_cards', 'total_red_cards', 'has_agent',
    'ga_per90min', 'contract_years_remaining'
]

# Các cột phân loại ít giá trị (Low Cardinality) -> Sẽ dùng One-Hot/Label
categorical_features_low = [
    'sub_position', 'foot'
]

# Các cột phân loại nhiều giá trị (High Cardinality) -> Sẽ dùng Target Encoding
categorical_features_high = [
    'current_club_name', 'country_of_citizenship',  'current_club_domestic_competition_id'
]

target = 'market_value_in_eur'

# Tạo dataframe gọn nhẹ
df_model = df[numeric_features + categorical_features_low + categorical_features_high + [target]].copy()

# Xử lý nhanh các giá trị null trong cột số (nếu có) bằng 0 hoặc trung bình
df_model[numeric_features] = df_model[numeric_features].fillna(0)

print("Kích thước dữ liệu ban đầu:", df_model.shape)
df_model.head()

Kích thước dữ liệu ban đầu: (21378, 17)


,age_at_last_season,height_in_cm,club_position,total_goals,total_assists,total_minutes_played,total_yellow_cards,total_red_cards,has_agent,ga_per90min,contract_years_remaining,sub_position,foot,current_club_name,country_of_citizenship,current_club_domestic_competition_id,market_value_in_eur
0,37.00,184.00,8.00,12,8,2429,6,0,1,0.74,3,Centre-Forward,right,Società Sportiva Lazio S.p.A.,Germany,serie-a,"1,000,000.00"
1,37.00,190.00,4.00,0,0,181,0,0,1,0.00,3,Goalkeeper,left,Borussia Dortmund,Germany,bundesliga,"750,000.00"
4,36.00,194.00,12.00,0,0,450,0,0,1,0.00,3,Goalkeeper,right,FC Bayern München,Germany,bundesliga,"100,000.00"
7,35.00,179.00,2.00,2,2,679,1,0,0,0.53,3,Attacking Midfield,both,Arsenal Football Club,Czech Republic,premier-league,"350,000.00"
9,34.00,193.00,8.00,1,0,221,0,0,0,0.41,8,Centre-Forward,right,Málaga CF,Paraguay,laliga,"250,000.00"


### One-hot encoding
Xử lý các biến phân loại ít giá trị, không phân thứ tự (sub_position, foot) bằng One-Hot Encoding. Chúng ta làm bước này trước khi chia tập train/test vì nó không gây rò rỉ dữ liệu

In [41]:
df_encoded = pd.get_dummies(df_model, columns=categorical_features_low, drop_first=True)

print("Kích thước sau khi One-Hot Encoding:", df_encoded.shape)
df_encoded.head()

Kích thước sau khi One-Hot Encoding: (21378, 29)


,age_at_last_season,height_in_cm,club_position,total_goals,total_assists,total_minutes_played,total_yellow_cards,total_red_cards,has_agent,ga_per90min,...,sub_position_Goalkeeper,sub_position_Left Midfield,sub_position_Left Winger,sub_position_Left-Back,sub_position_Right Midfield,sub_position_Right Winger,sub_position_Right-Back,sub_position_Second Striker,foot_left,foot_right
0,37.00,184.00,8.00,12,8,2429,6,0,1,0.74,...,False,False,False,False,False,False,False,False,False,True
1,37.00,190.00,4.00,0,0,181,0,0,1,0.00,...,True,False,False,False,False,False,False,False,True,False
4,36.00,194.00,12.00,0,0,450,0,0,1,0.00,...,True,False,False,False,False,False,False,False,False,True
7,35.00,179.00,2.00,2,2,679,1,0,0,0.53,...,False,False,False,False,False,False,False,False,False,False
9,34.00,193.00,8.00,1,0,221,0,0,0,0.41,...,False,False,False,False,False,False,False,False,False,True


### Chia train test và log transform cho target
Chia dữ liệu trước khi làm Target Encoding để tránh lỗi data leakage. Đồng thời, áp dụng `np.log1p` cho biến mục tiêu để xử lý vấn đề lệch dữ liệu (skewness) của giá trị cầu thủ.

In [42]:
# Tách Features (X) và Target (y)
X = df_encoded.drop(columns=[target])
y = df_encoded[target]

# 2. Chia Train/Test (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Log Transform biến mục tiêu
# Dùng log1p (log(1+x)) để tránh lỗi log(0) và chuẩn hóa phân phối
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

Train size: (17102, 28), Test size: (4276, 28)


### Target encoding
Tính trung bình giá trị (đã log) của biến mục tiêu cho từng nhóm trong tập Train, sau đó map sang tập Test. Những giá trị lạ ở tập Test (chưa từng thấy ở Train) sẽ được điền bằng giá trị trung bình toàn cục.

In [43]:
# Hàm Target Encoding đơn giản và hiệu quả
def apply_target_encoding(train_df, test_df, target_series, cols_to_encode):
    # Tạo copy để không ảnh hưởng dữ liệu gốc
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    # Tạo dataframe tạm nối X_train và y_train để tính toán
    temp_train = train_df.copy()
    temp_train['target_temp'] = target_series
    
    # Tính trung bình toàn cục (để điền cho các giá trị lạ ở tập test)
    global_mean = target_series.mean()
    
    for col in cols_to_encode:
        # Tính trung bình target cho mỗi giá trị trong cột
        encoding_map = temp_train.groupby(col)['target_temp'].mean()
        
        # Map vào tập train và test
        train_encoded[col] = train_encoded[col].map(encoding_map)
        test_encoded[col] = test_encoded[col].map(encoding_map)
        
        # Xử lý các giá trị bị thiếu (do xuất hiện ở test nhưng không có ở train)
        train_encoded[col] = train_encoded[col].fillna(global_mean)
        test_encoded[col] = test_encoded[col].fillna(global_mean)
        
    return train_encoded, test_encoded

# Áp dụng hàm
X_train_final, X_test_final = apply_target_encoding(
    X_train, 
    X_test, 
    y_train_log, 
    categorical_features_high
)

print("Đã hoàn thành Target Encoding!")
X_train_final[categorical_features_high].head()

Đã hoàn thành Target Encoding!


,current_club_name,country_of_citizenship,current_club_domestic_competition_id
17364,13.38,13.41,13.67
15693,12.23,13.09,12.98
25262,14.68,13.06,13.52
1184,12.26,12.95,12.78
27589,13.01,14.44,12.78


### Lưu tập train/test

In [44]:

train_data = X_train_final.copy()
train_data['market_value_log'] = y_train_log

test_data = X_test_final.copy()
test_data['market_value_log'] = y_test_log

# Lưu ra file CSV (không lưu index để tránh cột thừa khi load lại)
train_data.to_csv('../data/train_data_encoded.csv', index=False)
test_data.to_csv('../data/test_data_encoded.csv', index=False)

print("Đã lưu xong:")
print("- train_data_encoded.csv")
print("- test_data_encoded.csv")

Đã lưu xong:
- train_data_encoded.csv
- test_data_encoded.csv
